In [ ]:
# standard library
import os

import pandas as pd
import seaborn as sns
import numpy as np
from highstreets.data import make_dataset as mhsd
from dotenv import load_dotenv, find_dotenv
import matplotlib.pyplot as plt

import numpyro
from numpyro.infer import MCMC, NUTS, Predictive, init_to_feasible
import numpyro.distributions as dist
from jax import random

# import jax.numpy as jnp
import arviz as az

import dill

assert numpyro.__version__.startswith("0.9.2")

load_dotenv(find_dotenv())

YOY_FILE = os.environ.get("YOY_FILE")
PROFILE_FILE = os.environ.get("PROFILE_FILE")
PROJECT_ROOT = os.environ.get("PROJECT_ROOT")

%load_ext autoreload
%autoreload 2

sns.set_theme(style="darkgrid")
sns.set_context("notebook")

### Load mastercard spend data along with high street profiles and setup data arrays and time vectors for convenience


In [ ]:
hsp = pd.read_excel(PROFILE_FILE)
hsd_yoy = pd.read_csv(YOY_FILE, parse_dates=["week_start"])

# some important dates
nb_dates = pd.to_datetime(
    [
        "2020-03-24",  # first lockdown starts
        "2020-06-15",  # shops reopen
        "2020-11-05",  # second lockdown starts
        "2020-12-02",  # back to 'tier 2' (i.e. partial reopening)
        "2021-01-05",  # third lockdown starts
        "2021-04-12",  # shops reopen
    ]
)

# average weekday and weekend expenditure (should probably relax this
# later - no need to lose information)
# hsd_yoy_minimal = mhsd.get_minimal_df(hsd_yoy, "yoy_").dropna(how="any", axis="rows")
# hsd_yoy_minimal = mhsd.avg_retail_wd_we(hsd_yoy, "yoy_").dropna(how="any", axis="rows")

hsd_yoy_minimal = mhsd.stack_retail_we_wd(hsd_yoy, "yoy_").dropna(
    how="any", axis="rows"
)

dates_2020 = ("2020-04-15", "2020-10-31")
dates_2020_full = ("2020-01-01", "2020-12-31")
dates_2021 = ("2021-02-12", "2021-08-31")
dates_full = ("2020-01-01", "2021-12-31")

data_2020 = mhsd.extract_data_array(hsd_yoy_minimal, dates_2020, "txn_amt")
data_2021 = mhsd.extract_data_array(hsd_yoy_minimal, dates_2021, "txn_amt")
data_2020_full = mhsd.extract_data_array(hsd_yoy_minimal, dates_2020_full, "txn_amt")
data_full = mhsd.extract_data_array(hsd_yoy_minimal, dates_full, "txn_amt")

start_times = {"2020": "2020-04-01", "2021": "2021-04-12", "full": "2020-04-01"}
tvecs = {"2020": data_2020.index, "2021": data_2021.index, "full": data_full.index}
arrays = {
    "2020": np.transpose(data_2020.to_numpy()),
    "2021": np.transpose(data_2021.to_numpy()),
    "full": np.transpose(data_full.to_numpy()),
}

### Set up data to be used in hierarcical regressions

In [ ]:
predictors = [
    "percentage of commercial addresses (%)",
    "total estimated number of home workers",
    "Sum_y2019_07wd",
]
full_data = (
    hsd_yoy_minimal.join(
        hsp[["highstreet_id"] + predictors],
        on="highstreet_id",
        how="left",
        lsuffix="_left",
        rsuffix="_right",
    )
    .drop(["highstreet_id_right", "txn_cnt"], axis=1)
    .rename(columns={"highstreet_id_left": "highstreet_id"})
)
full_data["weeks_since_start"] = (
    full_data.index - pd.to_datetime(nb_dates[0])
) / pd.Timedelta(1, "W")
train = full_data.loc[nb_dates[0] : nb_dates[1]]
train.head()

### Define hierarchical regression model

In [ ]:
def model(highstreet_id, weeks, hs_obs=None):
    mu_a = numpyro.sample("mu_a", dist.Normal(0.0, 1.0))
    sigma_a = numpyro.sample("sigma_a", dist.HalfNormal(1.0))
    mu_b = numpyro.sample("mu_b", dist.Normal(0.0, 1.0))
    sigma_b = numpyro.sample("sigma_b", dist.HalfNormal(1.0))

    unique_hs_ids = np.unique(highstreet_id)
    n_hs = len(unique_hs_ids)

    with numpyro.plate("plate_i", n_hs):
        a = numpyro.sample("a", dist.Normal(mu_a, sigma_a))
        b = numpyro.sample("b", dist.Normal(mu_b, sigma_b))

    sigma = numpyro.sample("sigma", dist.HalfNormal(1.0))
    hs_est = a[highstreet_id] + b[highstreet_id] * weeks

    with numpyro.plate("data", len(highstreet_id)):
        numpyro.sample("obs", dist.Normal(hs_est, sigma), obs=hs_obs)

In [ ]:
hs_obs = train["txn_amt"].values
weeks = train["weeks_since_start"].values
highstreet_id = train["highstreet_id"].values

graph = numpyro.render_model(
    model, model_args=(highstreet_id, weeks), render_distributions=True
).unflatten()

graph.render(
    filename=PROJECT_ROOT
    + "/reports/figures/hierarchical_model_graphs/simple_regression_2020_pooled"
)

graph

In [ ]:
# def model(highstreet_id, weeks, predictors, hs_obs=None):
#     mu_gamma_a = numpyro.sample(
#         "mu_gamma_a",
#         dist.MultivariateNormal(
#             loc=0.0,
#             covariance_matrix=jnp.eye(predictors.shape[0]),
#         ),
#     )
#     sigma_a = numpyro.sample("sigma_a", dist.HalfNormal(1.0))
#     mu_gamma_b = numpyro.sample("mu_gamma_b", dist.MultivariateNormal(0.0))
#     sigma_b = numpyro.sample("sigma_b", dist.HalfNormal(1.0))

#     unique_hs_ids = np.unique(highstreet_id)
#     n_hs = len(unique_hs_ids)

#     with numpyro.plate("plate_i", n_hs):
#         gamma_a = numpyro.sample(
#             "gamma_a",
#             dist.MultivariateNormal(
#                 loc=mu_gamma_a,
#                 covariance_matrix=sigma_a * jnp.eye(),
#             ),
#         )
#         b = numpyro.sample("b", dist.Normal(mu_b, sigma_b))

#     sigma = numpyro.sample("sigma", dist.HalfNormal(1.0))
#     hs_est = a[highstreet_id] + b[highstreet_id] * weeks

#     with numpyro.plate("data", len(highstreet_id)):
#         numpyro.sample("obs", dist.Normal(hs_est, sigma), obs=hs_obs)

### Sample from the posterior

In [ ]:
nuts_kernel = NUTS(model, init_strategy=init_to_feasible())

mcmc = MCMC(nuts_kernel, num_samples=2000, num_warmup=1000)
rng_key = random.PRNGKey(0)
mcmc.run(rng_key, highstreet_id, weeks, hs_obs=hs_obs)
posterior_samples = mcmc.get_samples()

#### Save the model samples

In [ ]:
dill_file = PROJECT_ROOT + "/models/bayesian/posterior_samples_basic.pkl"
with open(dill_file, "wb") as f:
    dill.dump(
        posterior_samples,
        f,
    )


dill_file = PROJECT_ROOT + "/models/bayesian/mcmc_basic.pkl"
with open(dill_file, "wb") as f:
    dill.dump(
        mcmc,
        f,
    )

### Load previous samples

In [ ]:
dill_file = PROJECT_ROOT + "models/bayesian/posterior_samples_basic.pkl"
with open(dill_file, "rb") as f:
    posterior_samples = dill.load(f)

dill_file = PROJECT_ROOT + "models/bayesian/mcmc_basic.pkl"
with open(dill_file, "rb") as f:
    mcmc = dill.load(f)

### Trace plot for centered model

In [ ]:
data = az.from_numpyro(mcmc)
az.plot_trace(data, compact=True, figsize=(14, 32))

### Examine model output

In [ ]:
mcmc.print_summary()

### Compare predictions to data

In [ ]:
weeks_pred = train[train["highstreet_id"] == 1]["weeks_since_start"]
pred_template = []
for i in np.unique(train["highstreet_id"]):
    df = pd.DataFrame(columns=["highstreet_id", "weeks"])
    df["weeks"] = weeks_pred
    df["highstreet_id"] = i
    pred_template.append(df)
pred_template = pd.concat(pred_template, ignore_index=True)

highstreet_id = pred_template["highstreet_id"].values
weeks = pred_template["weeks"].values
predictive = Predictive(model, posterior_samples, return_sites=["sigma", "obs"])
samples_predictive = predictive(random.PRNGKey(0), highstreet_id, weeks, None)


df_act_pred = pd.DataFrame(
    columns=["highstreet_id", "weeks_since_start", "txn_amt_pred", "sigma"]
)
df_act_pred["highstreet_id"] = pred_template["highstreet_id"]
df_act_pred["weeks_since_start"] = pred_template["weeks"]
df_act_pred["txn_amt_pred"] = samples_predictive["obs"].T.mean(axis=1)
df_act_pred["sigma"] = samples_predictive["obs"].T.std(axis=1)
df_act_pred["hs_inf"] = df_act_pred["txn_amt_pred"] - df_act_pred["sigma"]
df_act_pred["hs_sup"] = df_act_pred["txn_amt_pred"] + df_act_pred["sigma"]
df_act_pred = pd.merge(
    df_act_pred,
    train[["highstreet_id", "weeks_since_start", "txn_amt"]],
    how="left",
    on=["highstreet_id", "weeks_since_start"],
)
df_act_pred = df_act_pred.rename(columns={"txn_amt": "txn_amt_true"})
df_act_pred.head()

In [ ]:
def chart_highstreet_with_predictions(highstreet_id, ax):
    data = df_act_pred[df_act_pred["highstreet_id"] == highstreet_id]
    x = data["weeks_since_start"]
    ax.set_title(highstreet_id)
    ax.plot(x, data["txn_amt_true"], "o")
    ax.plot(x, data["txn_amt_pred"])
    ax = sns.regplot(
        x=x, y=data["txn_amt_true"], ax=ax, ci=None, line_kws={"color": "red"}
    )
    ax.fill_between(x, data["hs_inf"], data["hs_sup"], alpha=0.5, color="#ffcd3c")
    ax.set_ylabel("MRLI")


f, axes = plt.subplots(1, 3, figsize=(15, 5))
hs_plot = [189, 310, 446]
for i, id in enumerate(hs_plot):
    chart_highstreet_with_predictions(id, axes[i])

In [ ]:
train

In [ ]:
def model_w_regressors(highstreet_id, weeks, hs_obs=None):
    mu_a = numpyro.sample("mu_a", dist.Normal(0.0, 1.0))
    sigma_a = numpyro.sample("sigma_a", dist.HalfNormal(1.0))
    mu_b = numpyro.sample("mu_b", dist.Normal(0.0, 1.0))
    sigma_b = numpyro.sample("sigma_b", dist.HalfNormal(1.0))

    unique_hs_ids = np.unique(highstreet_id)
    n_hs = len(unique_hs_ids)

    with numpyro.plate("plate_i", n_hs):
        a = numpyro.sample("a", dist.Normal(mu_a, sigma_a))
        b = numpyro.sample("b", dist.Normal(mu_b, sigma_b))

    sigma = numpyro.sample("sigma", dist.HalfNormal(1.0))
    hs_est = a[highstreet_id] + b[highstreet_id] * weeks

    with numpyro.plate("data", len(highstreet_id)):
        numpyro.sample("obs", dist.Normal(hs_est, sigma), obs=hs_obs)